## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
#if working on local system we need to start spark session before doing any operations...
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName('Basics').getOrCreate()

# File location and type
file_location = "/FileStore/tables/people.json"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

age,name
null,Michael
30,Andy
19,Justin


In [3]:
# Create a view or table

temp_table_name = "people_json"

df.createOrReplaceTempView(temp_table_name)

In [4]:
%sql

/* Query the created temp table in a SQL cell */

select * from `people_json`

age,name
null,Michael
30,Andy
19,Justin


In [5]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "people_json"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [6]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [7]:
df.printSchema()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [8]:
df.columns

Out[6]: ['age', 'name']

In [9]:
df.describe().show()

+-------+------------------+-------+
summary| age| name|
+-------+------------------+-------+
 count| 2| 3|
 mean| 24.5| null|
 stddev|7.7781745930520225| null|
 min| 19| Andy|
 max| 30|Michael|
+-------+------------------+-------+

In [10]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [11]:
# To change the datatype of the schema... here we are changing 'age' datatype from long to integer
data_schema = [StructField('age', IntegerType(), True), 
               StructField('name', StringType(), True)]

In [12]:
final_struc = StructType(fields = data_schema)

In [13]:
df = spark.read.json('/FileStore/tables/people.json', schema = final_struc)

In [14]:
df.printSchema()

root
-- age: integer (nullable = true)
-- name: string (nullable = true)

In [15]:
df.select('age').show()

+----+
 age|
+----+
null|
 30|
 19|
+----+

In [16]:
df.head(2)

Out[25]: [Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [17]:
df.select(['age', 'name']).show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [18]:
new_df = df.withColumn('double_age', df['age']*2)

In [19]:
new_df.show()

+----+-------+----------+
 age| name|double_age|
+----+-------+----------+
null|Michael| null|
 30| Andy| 60|
 19| Justin| 38|
+----+-------+----------+

In [20]:
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
my_new_age| name|
+----------+-------+
 null|Michael|
 30| Andy|
 19| Justin|
+----------+-------+

In [21]:
df.createOrReplaceTempView('people')

In [22]:
results = spark.sql("select * from people where age = 30")

In [23]:
results.show()

+---+----+
age|name|
+---+----+
 30|Andy|
+---+----+